In [1]:
# PyVisa imports

import pyvisa
from pyvisa.errors import VisaIOError
from datetime import datetime
import time
from statistics import mean 

#Influx imports
from dotenv import load_dotenv
import os
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client import InfluxDBClient
import random
from datetime import datetime
from time import sleep
import logging
from influxdb_client import InfluxDBClient, BucketRetentionRules, BucketsApi

#Pyside6 imports
from PySide6.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QLineEdit, QVBoxLayout, QHBoxLayout, QWidget, QMessageBox, QListWidget, QTextEdit, QFileDialog
from PySide6.QtCore import Qt, QTimer
import serial
import serial.tools.list_ports
import sys
from datetime import datetime
import csv

# E36312A Code

In [2]:
# Variables

list_devices = False # If you want to list devices at start set this to true

In [3]:
# Functions
class E36312A:
    
    def test(self, DPS):
        """Tests device connection"""
        DPS.read_termination = '\n'
        DPS.write_termination = '\n'
        DPS.channels = 3
        DPS.powertest = 13
        DPS.datalog = 30
        DPS.write("*CLS")
        DPS.write("*RST")
        powertest = []
        for state in ['1', '0']:
            for channel in range(DPS.channels):
                powertest.append(DPS.write(f"OUTP {state}, (@{channel+1})"))
        if (not mean(powertest) == DPS.powertest):
            print("Power Test Failed")
            return None
    
        datalog = []
        for state in ['1', '0']:
            for type in ['VOLT', 'CURR']:
                datalog.append(DPS.write(f"SENS:DLOG:FUNC:{type} {state}, (@1:3)"))
        if (not mean(datalog) == DPS.datalog):
            print("Data Logger Test Failed")
            return None
        return True
            
    def turnOn(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
        else:
            DPS.write(f"OUTP 1, (@{ch})")
    
    def turnOff(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
        else:
            DPS.write(f"OUTP 0, (@{ch})")

    def turnAllOn(self, DPS):
        chlist=[1,2,3]
        for ch in chlist:
            DPS.write(f"OUTP 1, (@{ch})")

    def turnAllOff(self, DPS):
        chlist=[1,2,3]
        for ch in chlist:
            DPS.write(f"OUTP 0, (@{ch})")

    def findVoltageRange(self, DPS, ch):
        if (ch == 1):  
            maximum = min(float(DPS.query(f"VOLT:PROT? (@{ch})")), 6.18)
        else:
            maximum = min(float(DPS.query(f"VOLT:PROT? (@{ch})")), 25.75)
        return 0, maximum

    def findCurrentRange(self, DPS, ch):
        if (ch == 1):  
            maximum = 5.15
        else:
            maximum = 1.03
        return 0.001, maximum
    
    def setVoltage(self, DPS, ch, voltage):
        range = self.findVoltageRange(DPS, ch)
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
            return -1
        elif voltage > range[1] or voltage < range[0]:
            #print("Voltage Out of Range")
            return -2
        else:
            DPS.write(f"VOLT {voltage}, (@{ch})")
            return 1

    def setCurrentLimit(self, DPS, ch, current):
        range = self.findCurrentRange(DPS, ch)
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
            return -1
        elif current > range[1] or current < range[0]:
            #print("Current Out of Range")
            return -2
        else:
            DPS.write(f"CURR {current}, (@{ch})")
            return 1
    
    def getCurrentLimit(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
            return None
        else:
            current = DPS.query(f"SOUR:CURR? (@{ch})")
            return current
    
    def getSetVoltage(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
            return None
        else:
            voltage = DPS.query(f"SOUR:VOLT? (@{ch})")
            return voltage
    
    def getCurrent(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
            return None
        else:
            current = DPS.query(f"MEAS:CURR:DC? (@{ch})")
            return current
    
    def getVoltage(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
            return None
        else:
            voltage = DPS.query(f"MEAS:VOLT:DC? (@{ch})")
            return voltage
    
    def enableDlogVoltage(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
        else:
            DPS.write(f"SENS:DLOG:FUNC:VOLT 1, (@{ch})")
    
    def disableDlogVoltage(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
        else:
            DPS.write(f"SENS:DLOG:FUNC:VOLT 0, (@{ch})")
    
    def enableDlogCurrent(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
        else:
            DPS.write(f"SENS:DLOG:FUNC:CURR 1, (@{ch})")
    
    def disableDlogCurrent(self, DPS, ch):
        chlist=[1,2,3]
        if int(ch) not in chlist:
            print("Invalid Channel")
        else:
            DPS.write(f"SENS:DLOG:FUNC:CURR 0, (@{ch})")
    
    def setDlogTime(self, DPS, time):
        DPS.write(f"SENS:DLOG:TIME {time}")
    
    def startDlog(self, DPS, file):
        DPS.write(f"INIT:DLOG 'External:/{file}.csv'")

    def writeCommand(self, DPS, command):
        try:
            response = DPS.query(command)
            return response
        except VisaIOError:
            return "Invalid Command"

## InfluxDB

In [4]:
class InfluxClient:
    def __init__(self, token, org, bucket): 
        self._org = org 
        self._bucket = bucket
        self._client = InfluxDBClient(url="http://bragi.caltech.edu:8086", token=token)

    def write_data(self, data, write_option=SYNCHRONOUS):
        write_api = self._client.write_api(write_option)
        try:
            write_api.write(self._bucket, self._org, data, write_precision='s')
            logging.info('Data written successfully')
        except Exception as e:
            logging.error(f'Failed to write data: {e}')

# GUI

## PySide6

In [22]:
def GUI_deviceSelection():
    selected_device = []
    selected_type = ""
    rm = pyvisa.ResourceManager()
    PyVisaIds = list(rm.list_resources())
    PySerialIds = []
    serialMap = {}
    allDevices = []
    skip = 0

    app = QApplication.instance()
    if app is None:
        app = QApplication([])
    else:
        #print("QApplication instance already exists.")
        app.quit()
        app = QApplication.instance()
       # print("QApplication instance no longer exists.")
    
    def selectIndex():
        nonlocal selected_device
        selected_indices = Lb.selectedIndexes()
        if selected_indices:
            # print(selected_type)
            selected_index = selected_indices[0].row()
            if (selected_type == "PyVisa"):
                selected_device.append("PyVisa")
                selected_device.append(PyVisaIds[selected_index])
            if (selected_type == "PySerial"):
                selected_device.append("PySerial")
                selected_device.append(serialMap[selected_index][0])
                selected_device.append(serialMap[selected_index][1])
            main.close()
        else:
            QMessageBox.warning(main, "No Device Selected", "No Device Selected", QMessageBox.Ok)

    def wait():
        pass

    def fillSelection(array):
        nonlocal selected_type
        max_length = 1
        Lb.clear() 
        for i in range (len(array)):
            if (array == PyVisaIds):
                selected_type = "PyVisa"
                device = rm.open_resource(array[i])
                device_id = device.query("*IDN?")
                Lb.addItem(device_id)
                max_length = max(max_length, len(device_id))
            if (array == PySerialIds):
                selected_type = "PySerial"
                BAUD = [4800, 9600, 19200, 38400, 57600, 115200, 230400, 460800, 921600]
                comlist = serial.tools.list_ports.comports()
                connected = []
                for element in comlist:
                    connected.append(element.device)
                    
                for com in connected:
                    for baud in BAUD:
                        try:
                            device = serial.Serial(com, baud, timeout=1)
                            device.write(b'*IDN?\n')
                            time.sleep(0.1)
                            device_id = device.read_all().decode("utf-8")
                            device.close()
                            if (device_id):
                                Lb.addItem(device_id)
                                max_length = max(max_length, len(device_id))
                                serialMap[i] = [com, baud]
                                break
                        except Exception as e:
                            print(e)
                            continue
    
    main = QWidget()
    main.setWindowTitle("Device Selection")
    layout = QVBoxLayout()

    Lb = QListWidget()
    max_length = 0
    deleteIndex = []
    for i in range (len(PyVisaIds)):
        try:
            device = rm.open_resource(PyVisaIds[i])
            device.query("*IDN?")
            allDevices.append(device)
        except VisaIOError:
            deleteIndex.append(i)
            #allDevices.append("NULL")

        
    for i in range (len(deleteIndex) - 1, -1, -1):
        PySerialIds.insert(0, PyVisaIds[deleteIndex[i]])
        del PyVisaIds[deleteIndex[i]]
        
    Lb.setMinimumWidth(max_length * 8)  # Adjust width to fit content
    layout.addWidget(Lb)

    btn_show_index = QPushButton("Select Device")
    btn_show_index.clicked.connect(selectIndex)
    layout.addWidget(btn_show_index)

    btn_select_pyvisa = QPushButton("Show PyVisa Devices")
    btn_select_pyvisa.clicked.connect(lambda: fillSelection(PyVisaIds))
    layout.addWidget(btn_select_pyvisa)

    btn_select_pyserial = QPushButton("Show PySerial Devices")
    btn_select_pyserial.clicked.connect(lambda: fillSelection(PySerialIds))
    layout.addWidget(btn_select_pyserial)
    
    main.setLayout(layout)
    main.show()
   
    app.exec()
    timer = QTimer()
    timer.timeout.connect(wait)
    timer.start(0)
    wait()
    return selected_device

class GUI_E36312A(QMainWindow):
    def __init__(self, DPS, parent=None): 
        global x
        global allButtons
        global upload_timer
        global buckets
        global bucketList
        global bucketNames
        global selectedBucket
        global frequency
        global token
        global org
        global buckets_api

        token = os.getenv('TOKEN')
        org = os.getenv('ORG')

        self.client = InfluxDBClient(url="http://bragi.caltech.edu:8086", token=token, org=org)
        
        # Initialize BucketsApi with the client
        buckets_api = BucketsApi(self.client)

        frequency = 1.0
        selectedBucket = "None"
        bucketList = buckets_api.find_buckets().buckets
        bucketNames = []
        buckets = QListWidget()
        allButtons = []
        x = E36312A()
        super().__init__(parent)
        self.DPS = DPS
        # DPS = init() # Initiates device
        init_test = x.test(DPS) # Tests device
        if init_test:
            pass
        else:
            print("Error in initiation or test")
        self.setWindowTitle("E36312A GUI")

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.main_layout = QHBoxLayout(self.central_widget)
        
        self.control_layout = QVBoxLayout()
        self.recording_layout = QVBoxLayout()

        self.channel_layout = QHBoxLayout()

        self.outputLabel = QLabel("Output Channels")
        self.control_layout.addWidget(self.outputLabel)

        # Create a channel for demonstration
        self.createChannel(1)
        self.createChannel(2)
        self.createChannel(3)

        self.control_layout.addLayout(self.channel_layout)

        self.allOnButton = QPushButton("Turn All On", clicked=lambda: self.toggleAllChannelsOn())
        self.control_layout.addWidget(self.allOnButton)

        self.allOffButton = QPushButton("Turn All Off", clicked=lambda: self.toggleAllChannelsOff())
        self.control_layout.addWidget(self.allOffButton)


        self.terminal_layout = QVBoxLayout()

        # Add Terminal section
        self.addTerminal()

        self.createNotesBox()
        self.importTextFile()
        self.notes_edit.textChanged.connect(self.saveNotes)

        self.control_layout.addLayout(self.terminal_layout)

        self.recordLabel = QLabel("Record to InfluxDB and Grafana")
        self.recording_layout.addWidget(self.recordLabel)

        self.bucketLabel = QLabel("Selected Bucket: None")
        self.recording_layout.addWidget(self.bucketLabel)

        self.currentFrequencyLabel = QLabel(f"Frequency of Measurements: {frequency}s")
        self.recording_layout.addWidget(self.currentFrequencyLabel)

        self.isRecording = QLabel("Status: Recording Stopped")
        self.recording_layout.addWidget(self.isRecording)
        
        self.addRecording(self.bucketLabel, self.isRecording)


        self.main_layout.addLayout(self.control_layout)
        self.main_layout.addLayout(self.recording_layout)        

        upload_timer = QTimer(self)
        upload_timer.timeout.connect(lambda: self.record(self.DPS))
    
    def createChannel(self, ch):
        global allButtons
        #x = E36312A()
        self.layout = QVBoxLayout()
        channel_label = QLabel(f"Channel {ch}")
        self.layout.addWidget(channel_label)

        button = QPushButton("OFF", clicked=lambda: self.toggleButton(ch, button))
        button.setStyleSheet("background-color: red")
        self.layout.addWidget(button)

        allButtons.append(button)

        voltage_range = x.findVoltageRange(self.DPS, ch)
        voltage_label = QLabel("Set Voltage")
        self.layout.addWidget(voltage_label)

        voltage_entry = QLineEdit()
        hint_voltage = f"{voltage_range[0]}V - {voltage_range[1]}V"
        voltage_entry.setPlaceholderText(hint_voltage)
        voltage_entry.setStyleSheet("color: black")
        voltage_entry.setStyleSheet("background-color: white")
        voltage_entry.installEventFilter(self)  # To handle placeholder text behavior
        self.layout.addWidget(voltage_entry)

        voltage_button = QPushButton("Set Voltage", clicked=lambda: self.readVoltageEntry(ch, voltage_entry))
        voltage_button.setStyleSheet("background-color: white")
        self.layout.addWidget(voltage_button)

        current_range = x.findCurrentRange(self.DPS, ch)
        current_label = QLabel("Set Current Limit")
        self.layout.addWidget(current_label)

        current_entry = QLineEdit()
        hint_current = f"{current_range[0]}A - {current_range[1]}A"
        current_entry.setPlaceholderText(hint_current)
        current_entry.setStyleSheet("color: black")
        current_entry.setStyleSheet("background-color: white")
        current_entry.installEventFilter(self)  # To handle placeholder text behavior
        self.layout.addWidget(current_entry)

        current_button = QPushButton("Set Current Limit", clicked=lambda: self.readCurrentEntry(ch, current_entry))
        current_button.setStyleSheet("background-color: white")

        self.layout.addWidget(current_button)

        # To display and update voltage and current readings:
        display_voltage = QLabel()
        self.layout.addWidget(display_voltage)

        display_current = QLabel()
        self.layout.addWidget(display_current)

        # update_timer = QTimer(self)
        # update_timer.timeout.connect(lambda: self.updateVoltageCurrent(ch, display_voltage, display_current))
        # update_timer.start(1000)

        self.widget = QWidget()
        self.widget.setLayout(self.layout)
        if (ch == 1):
            self.widget.setStyleSheet("background-color: rgba(255, 255, 0, 64);")
        if (ch == 2):
            self.widget.setStyleSheet("background-color: rgba(0, 128, 0, 64);")
        if (ch == 3):
            self.widget.setStyleSheet("background-color: rgba(0, 0, 255, 64);")


        self.channel_layout.addWidget(self.widget)

    def toggleButton(self, ch, button):
        #x = E36312A()
        if button.text() == 'OFF':
            x.turnOn(self.DPS, ch)
            button.setText('ON')
            button.setStyleSheet("background-color: green")
        else:
            x.turnOff(self.DPS, ch)
            button.setText('OFF')
            button.setStyleSheet("background-color: red")

    def toggleAllChannelsOn(self):
        global x
        x.turnAllOn(self.DPS)
        for i in range(3):  # Assuming you have 3 channels
            allButtons[i].setText("ON")
            allButtons[i].setStyleSheet("background-color: green")

    def toggleAllChannelsOff(self):
        x.turnAllOff(self.DPS)
        for i in range(3):  # Assuming you have 3 channels
            allButtons[i].setText("OFF")
            allButtons[i].setStyleSheet("background-color: red")
            
    def readVoltageEntry(self, ch, voltage_entry):
        #x = E36312A()
        text = voltage_entry.text()
        voltage_entry.clear()
        if text == "" or text.startswith("Set Voltage"):
            QMessageBox.warning(self, "Warning", "No Input", QMessageBox.Ok)
            return
        try:
            voltage = float(text)
            result = x.setVoltage(self.DPS, ch, voltage)
            if result == -2:
                QMessageBox.warning(self, "Warning", "Voltage Entered Out Of Range", QMessageBox.Ok)
        except ValueError:
            pass

    def readCurrentEntry(self, ch, current_entry):
        #x = E36312A()
        text = current_entry.text()
        current_entry.clear()
        if text == "" or text.startswith("Set Current Limit"):
            QMessageBox.warning(self, "Warning", "No Input", QMessageBox.Ok)
            return
        try:
            current = float(text)
            result = x.setCurrentLimit(self.DPS, ch, current)
            if result == -2:
                QMessageBox.warning(self, "Warning", "Current Entered Out Of Range", QMessageBox.Ok)
        except ValueError:
            pass

    def updateVoltageCurrent(self, ch, display_voltage, display_current):
        #x = E36312A()
        voltage = float(x.getVoltage(self.DPS, ch))
        display_voltage.setText(f"{voltage} V")

        current = float(x.getCurrent(self.DPS, ch))
        display_current.setText(f"{current} A")

        csv_file = f'E36312A_channel{ch}.csv'
        with open(csv_file, 'a', newline='') as f:
            writer = csv.writer(f)
            row = (datetime.datetime.now(), voltage, current, voltage*current)
            writer.writerow(row)

    def addTerminal(self):
        self.layout = QVBoxLayout()
        terminal_label = QLabel("Terminal")
        self.layout.addWidget(terminal_label)

        self.terminal_input = QLineEdit()
        self.terminal_input.setPlaceholderText("Enter command")
        self.terminal_input.setStyleSheet("color: black")
        self.layout.addWidget(self.terminal_input)

        terminal_button = QPushButton("Send Command", clicked=self.sendTerminalCommand)
        self.layout.addWidget(terminal_button)

        self.terminal_output = QTextEdit()
        self.terminal_output.setReadOnly(True)
        self.layout.addWidget(self.terminal_output)
        self.terminal_layout.addLayout(self.layout)

    def sendTerminalCommand(self):
        command = self.terminal_input.text()
        self.terminal_input.clear()
        if command:
            response = x.writeCommand(self.DPS, command)
            self.terminal_output.append(f"Command: {command}\nResponse: {response}\n")

    
    def createNotesBox(self):
        self.layout = QVBoxLayout()
        notes_label = QLabel("Notes")
        self.layout.addWidget(notes_label)

        self.notes_edit = QTextEdit()
        self.layout.addWidget(self.notes_edit)
        self.terminal_layout.addLayout(self.layout)

    def importTextFile(self):
        file_path = "E36312A_Notes.txt"

        try:
            with open(file_path, 'r') as file:
                text = file.read()
            self.notes_edit.setPlainText(text)
        except FileNotFoundError:
            QMessageBox.warning(self, "File Not Found", f"File '{file_path}' not found.", QMessageBox.Ok)

    def saveNotes(self):
        file_path = "E36312A_Notes.txt"
        try:
            with open(file_path, 'w') as file:
                text = self.notes_edit.toPlainText()
                file.write(text)
        except IOError:
            QMessageBox.warning(self, "Error", f"Failed to save file '{file_path}'", QMessageBox.Ok)

    def startRecording(self, label):
        global upload_timer
        if (selectedBucket == "None"):
            QMessageBox.warning(label, "No Bucket Selected", "No Bucket Selected", QMessageBox.Ok)
        else:
            if not upload_timer.isActive():
                upload_timer.start(frequency * 1000)
                label.setText("Status: Recording Started")

        
    def stopRecording(self, label):
        global upload_timer
        upload_timer.stop()
        label.setText("Status: Recording Stopped")

    def setRecordingDelay(self, time, label, textbox): #when updating delay, the first measurement using the new delay always is 1 second longer, then it goes back to the correct set delay (not sure why)
        global frequency
        try:
            frequency = float(time)
            label.setText(f"Frequency of Measurements: {frequency}s")
            textbox.clear()

            if upload_timer.isActive():
                upload_timer.stop()
                upload_timer.start(frequency * 1000)
            else:
                upload_timer.stop()
        except:
            pass
        
    def record(self, DPS):
        IC = InfluxClient(token, org, selectedBucket)
        timestamp = int(datetime.now().timestamp())
        try:
            voltage1 = float(DPS.query("MEAS:VOLT:DC? (@1)").strip())
            current1 = float(DPS.query("MEAS:CURR:DC? (@1)").strip())
            voltage2 = float(DPS.query("MEAS:VOLT:DC? (@2)").strip())
            current2 = float(DPS.query("MEAS:CURR:DC? (@2)").strip())
            voltage3 = float(DPS.query("MEAS:VOLT:DC? (@3)").strip())
            current3 = float(DPS.query("MEAS:CURR:DC? (@3)").strip())
        except VisaIOError as e:
            logging.error(f'Failed to read from GPIB device: {e}')
    
        # Format data for InfluxDB
        voltage_data1 = f"E36312A,Channel=1 voltage={voltage1} {timestamp}"
        current_data1 = f"E36312A,Channel=1 current={current1} {timestamp}"
        voltage_data2 = f"E36312A,Channel=2 voltage={voltage2} {timestamp}"
        current_data2 = f"E36312A,Channel=2 current={current2} {timestamp}"
        voltage_data3 = f"E36312A,Channel=3 voltage={voltage3} {timestamp}"
        current_data3 = f"E36312A,Channel=3 current={current3} {timestamp}"
        
        # Write data to InfluxDB
        if (DPS.query("OUTP:STAT? (@1)")):
            IC.write_data(voltage_data1, write_option=SYNCHRONOUS)
            IC.write_data(current_data1, write_option=SYNCHRONOUS)
        if (DPS.query("OUTP:STAT? (@2)")):
            IC.write_data(voltage_data2, write_option=SYNCHRONOUS)
            IC.write_data(current_data2, write_option=SYNCHRONOUS)
        if (DPS.query("OUTP:STAT? (@3)")):
            IC.write_data(voltage_data3, write_option=SYNCHRONOUS)
            IC.write_data(current_data3, write_option=SYNCHRONOUS)

    def addRecording(self, bucketLabel, recordingLabel):
        self.layout = QVBoxLayout()
        self.startRecordingButton = QPushButton("Start Recording", clicked=lambda: self.startRecording(recordingLabel))
        self.layout.addWidget(self.startRecordingButton)
        
        self.stopRecordingButton = QPushButton("Stop Recording", clicked=lambda: self.stopRecording(recordingLabel))
        self.layout.addWidget(self.stopRecordingButton)

        self.frequencyLabel = QLabel("Set Frequency of Measurements")
        self.layout.addWidget(self.frequencyLabel)
        self.frequencyEntry = QLineEdit()
        self.frequencyEntry.setPlaceholderText("Enter time in seconds")
        self.layout.addWidget(self.frequencyEntry)
        self.frequencyButton = QPushButton("Set Frequency", clicked=lambda: self.setRecordingDelay(self.frequencyEntry.text(), self.currentFrequencyLabel, self.frequencyEntry))
        self.layout.addWidget(self.frequencyButton)

        self.newBucketLabel = QLabel("Create New Bucket")
        self.layout.addWidget(self.newBucketLabel)
        self.newBucketEntry = QLineEdit()
        self.newBucketEntry.setPlaceholderText("Enter bucket name")
        self.layout.addWidget(self.newBucketEntry)
        self.newBucketButton = QPushButton("Create Bucket", clicked=lambda: self.createBucket(self.newBucketEntry))
        self.layout.addWidget(self.newBucketButton)

        self.bucketSelectButton = QPushButton("Select Bucket", clicked=lambda: self.selectBucket(bucketLabel))
        self.layout.addWidget(self.bucketSelectButton)

        self.bucketsLabel = QLabel("Available Buckets")
        self.layout.addWidget(self.bucketsLabel)

        self.recording_layout.addLayout(self.layout)

        self.displayBuckets()

        self.layout.addWidget(buckets)

        self.recording_layout.addLayout(self.layout)
    
    def displayBuckets(self):
        global buckets
        global bucketList
        bucketList = buckets_api.find_buckets().buckets
        buckets.clear()
        bucketNames.clear()
        for bucket in bucketList:
            buckets.addItem(bucket.name)
            bucketNames.append(bucket.name)

    def selectBucket(self, bucketLabel): #when new bucket is created, the available indicies remain the same and any new indicies is refered back to 0
        global selectedBucket
        selectedIndices = buckets.selectedIndexes()
        if selectedIndices:
            ind = selectedIndices[0].row()
            selectedBucket = bucketNames[ind]
            print(ind)
            print(selectedBucket)
            bucketLabel.setText(f"Selected Bucket: {selectedBucket}")

    def createBucket(self, bucketNameEntry):
        bucketName = bucketNameEntry.text()
        bucketNameEntry.clear()
        
        if (bucketName not in bucketNames):
            retention_rules = BucketRetentionRules(type="expire", every_seconds=0)
            buckets_api.create_bucket(bucket_name=bucketName,
                                               retention_rules=retention_rules,
                                               org=org)
            self.displayBuckets()
        else:
            QMessageBox.warning(self, "Warning", "Bucket Already Exists", QMessageBox.Ok)
        
class GUI_1620A(QMainWindow):
    def __init__(self, hygrometer, parent=None):
        super().__init__(parent)
        self.hygrometer = hygrometer
        self.setWindowTitle("E36312A GUI")

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout(self.central_widget)
        self.label = QLabel("Test")
        self.layout.addWidget(self.label)

## GUI Start

In [23]:
def GUI_start():
    selected_device = GUI_deviceSelection()
    if not selected_device:
        sys.exit()

    rm = pyvisa.ResourceManager()
    if (selected_device[0] == "PyVisa"):
        my_device = rm.open_resource(selected_device[1])
        id = my_device.query("*IDN?").split(",")
    if (selected_device[0] == "PySerial"):
        my_device = serial.Serial(selected_device[1], selected_device[2], timeout=1)
        my_device.write(b'*IDN?\n')
        time.sleep(0.1)
        id = my_device.read_all().decode("utf-8").split(",")
        #print(id)
    class_name = f"GUI_{id[1]}"
    if hasattr(sys.modules[__name__], class_name):
        try:
            app = QApplication.instance()
            if app is None:
                app = QApplication(sys.argv)
            else:
                #print("QApplication instance already exists.")
                app.quit()
                app = QApplication.instance()
               # print("QApplication instance no longer exists.")
    
            main_window = globals()[class_name](my_device)
            main_window.show()
            app.exec()
        except Exception as e:
            app.quit()
            my_device.close()
            sys.exit()

In [27]:
GUI_start()

0
12345
0
12345


In [8]:
%tb

No traceback available to show.
